In [14]:
def count_repeat(seq):
    count_repeat_A = "A" * 8
    count_repeat_T = "T" * 8
    count_repeat_G = "G" * 8
    count_repeat_C = "C" * 8
    if count_repeat_A in seq or count_repeat_T in seq or count_repeat_G in seq or count_repeat_C in seq:
        return True

In [15]:
#test

seq1 = "CCCCCCCCCCCCC"
seq2 = "AAAAA"
    
count_repeat(seq1), count_repeat(seq2)

(True, None)

In [16]:
count_repeat(seq2)

In [18]:
#test
trial = [('AAAAAAAAAA', 'ATGCATGCAT'),
  ('AAAAAAAAAT', 'TGCATGCATG'),
  ('AAAAAAAATG', 'GCATGCATGC'),
  ('AAAAAAATGC', 'CATGCATGCA'),
  ('AAAAAATGCA', 'ATGCATGCAT'),
  ('AAAAATGCAT', 'AAAAAAAAAA'),
  ('AAAATGCATG', 'GCATGCATGC')]

for (i,j) in trial:
    trail2 = []
    if count_repeat(i) != True and count_repeat(j) != True:
        trail2.append((i,j))
        print(trail2)

[('AAAAAAATGC', 'CATGCATGCA')]
[('AAAAAATGCA', 'ATGCATGCAT')]
[('AAAATGCATG', 'GCATGCATGC')]


In [124]:
def probes(seq, extension_probe_length, ligation_probe_length, insert_size):

   
    # extension_probe_length = 18bp (ideally: 16-20bp)
    # ligation_probe_length = 22bp (ideally: 20-24bp)
    # insert_size = 112 nts
    extension_probe = []
    ligation_probe = []
    
    #all positions are zero-based
    extension_probe_pos =[]
    ligation_probe_pos = []
    target_seq = [] 

    # selecting probes of defined length iteratively from 5'-3' of the input sequence 
    for i in range(len(seq)):
        for j in range(i + 1, len(seq) + 1):
            if len(seq[i:j]) == extension_probe_length and len(seq[j+insert_size:j+insert_size+ligation_probe_length]) == ligation_probe_length:
                # discard sequences with consecutive polyNs (=repeats)
                if count_repeat(seq[i:j]) == True or count_repeat(seq[j+insert_size:j+insert_size+ligation_probe_length]) == True:
                    break
                
                # if sequences do not contain consecutive polyNs, proceed
                else:
                    # extract extension_probe of required length and store in a list
                    extension_probe.append(seq[i:j])
                    
                    # note start and end indexes of extension_probe 
                    extension_probe_pos.append(f"start={i}")
                    extension_probe_pos.append(f"end={j}")
                    
                    # skip 112 nts (=insert size) and extract following ligation_probe of required length
                    ligation_probe.append(seq[j+insert_size:j+insert_size+ligation_probe_length])
                    
                    # note start and end indexes of extension_probe and store in a list
                    ligation_probe_pos.append(f"start={j+insert_size}")
                    ligation_probe_pos.append(f"end={j+insert_size+ligation_probe_length}")
                      
                
                
    all_probes = list(zip(extension_probe,ligation_probe)) # combine lists 
    
             
    return all_probes, extension_probe_pos, ligation_probe_pos


def check_probes(all_probes):
    import Bio.SeqUtils 
    from Bio.SeqUtils import MeltingTemp as tm
    
    new_probes = []
    final_probes = []
    melting_temp = []
    
    # check for GC content of all_probes (expected range: 40-70%)
    for (i,j) in all_probes:
        if Bio.SeqUtils.GC(i) >= 40 or Bio.SeqUtils.GC(i) <= 70 and Bio.SeqUtils.GC(i) >= 40 or Bio.SeqUtils.GC(i) <= 70:
            # keep only those which follows the above criteria
            new_probes.append((i,j))
            
    # check for tm = melting temperature of all_probes 
    for (i,j) in new_probes:
        if tm.Tm_Wallace(i) >= 60 and tm.Tm_Wallace(j) >= 60:
            # keep only those which follows the above criteria
            final_probes.append((i,j))
            melting_temp.append(tm.Tm_Wallace(i))
            melting_temp.append(tm.Tm_Wallace(j))
                
    return final_probes
    

In [125]:
pwd

'/Users/priyalakra/Desktop/TERT_datasets'

In [126]:
seq = 'ATGTCACTGTATCGCCCTCTAGTTCTGCTGTCTTGTCTCTCATGGCCGCTGGCTGGCTTTTCTGCCACCGCGCTGACCAACCTCGTCGCGGAACCATTCGCTAAACTCGAACAGGACTTTGGCGGCTCCATCGGTGTGTACGCGATGGATACCGGCTCAGGCGCAACTGTAAGTTACCGCGCTGAGGAGCGCTTCCCACTGTGCAGCTCATTCAAGGGCTTTCTTGCTGCCGCTGTGCTGGCTCGCAGCCAGCAGCAGGCCGGCTTGCTGGACACACCCATCCGTTACGGCAAAAATGCGCTGGTTCCGTGGTCACCCATCTCGGAAAAATATCTGACAACAGGCATGACGGTGGCGGAGCTGTCCGCGGCCGCCGTGCAATACAGTGATAACGCCGCCGCCAATTTGTTGCTGAAGGAGTTGGGCGGCCCGGCCGGGCTGACGGCCTTCATGCGCTCTATCGGCGATACCACGTTCCGTCTGGACCGCTGGGAGCTGGAGCTGAACTCCGCCATCCCAGGCGATGCGCGCGATACCTCATCGCCGCGCGCCGTGACGGAAAGCTTACAAAAACTGACACTGGGCTCTGCACTGGCTGCGCCGCAGCGGCAGCAGTTTGTTGATTGGCTAAAGGGAAACACGACCGGCAACCACCGCATCCGCGCGGCGGTGCCGGCAGACTGGGCAGTCGGAGACAAAACCGGAACCTGCGGAGTGTATGGCACGGCAAATGACTATGCCGTCGTCTGGCCCACTGGGCGCGCACCTATTGTGTTGGCCGTCTACACCCGGGCGCCTAACAAGGATGACAAGCACAGCGAGGCCGTCATCGCCGCTGCGGCTAGACTCGCGCTCGAGGGATTGGGCGTCAACGGGCAGTAA'

In [127]:
all_probes, extension_probe_pos, ligation_probe_pos = probes(seq, extension_probe_length = 18, ligation_probe_length = 20, insert_size = 112)

In [128]:
len(all_probes)

733

In [130]:
final_probes = check_probes(all_probes)

In [131]:
len(final_probes)

258

In [26]:
# Preseting as table

import pandas as pd

def as_table(list_of_sequences, extension_probe_pos, ligation_probe_pos):
    data = pd.DataFrame(all_probes, extension_probe_pos, ligation_probe_pos, columns = ['Extension probe', 'Extension_probe_pos', 'Ligation probe', 'Extension_probe_pos'])
    return data
    pass

In [ ]:
def make_MIP(final_probes, linker):
    
    mips = []
    
    for (i,j) in final_probes:
        mip = j + linker + i
        mips.append(mip)
    return mips

    pass
        

In [ ]:
def open_folder(path):
    import os
    from Bio import SeqIO
    work_dir = os.chdir(path)
    
    seq = ''
    
    for file in os.listdir(work_dir):
        if file.endswith(".fna") or file.endswith(".fasta"):
            with open(file, 'r') as handle:
                handle.seek(0)
                
                for record in SeqIO.parse(handle, "fasta"):
                    
    pass
              